<a href="https://colab.research.google.com/github/Ananyalakshmi-P/Pythonprogramming/blob/main/horsehuman_images_classify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import urllib.request
import zipfile

In [32]:
import requests
url = "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip"
r = requests.get(url, allow_redirects=True)
with open("./horse-or-human.zip", 'wb') as fl:
  fl.write(r.content)

In [17]:
url = "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip"
filename = "horse-or-human.zip"
training_dir = "./horse-or-human/training/"
urllib.request.urlretrieve(url, filename)
zipref = zipfile.ZipFile(filename, 'r')
zipref.extractall(training_dir)
zipref.close()

HTTPError: ignored

In [35]:
filename = "/content/drive/MyDrive/archive (3).zip"
training_dir = "./horse-or-human/training/"
zipref = zipfile.ZipFile(filename, 'r')
zipref.extractall(training_dir)
zipref.close()

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [36]:
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

In [37]:
train_datagen = ImageDataGenerator(
rescale=1./255,
rotation_range=40,
width_shift_range=0.2,
height_shift_range=0.2,
shear_range=0.2,
zoom_range=0.2,
horizontal_flip=True,
fill_mode='nearest'
)

In [38]:
train_generator = train_datagen.flow_from_directory(
training_dir,
batch_size = 100,
target_size=(150, 150),
class_mode='binary'
)

Found 2566 images belonging to 1 classes.


In [39]:
import tensorflow as tf
model = tf.keras.models.Sequential([
tf.keras.layers.Conv2D(16, (3,3), activation='relu' ,
input_shape=(150, 150, 3)),
tf.keras.layers.MaxPooling2D(2, 2),
tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Flatten(),
tf.keras.layers.Dense(512, activation='relu'),
tf.keras.layers.Dense(1, activation='sigmoid')
])

In [40]:
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001), metrics=['accuracy'])

In [43]:
import requests
url = "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip"
r = requests.get(url, allow_redirects=True)
with open("./horse-or-human.zip", 'wb') as fl:
  fl.write(r.content)

In [46]:
validation_url = "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip"
validation_file_name = "/content/drive/MyDrive/validation.zip"
validation_dir = 'horse-or-human/validation/'
urllib.request.urlretrieve(validation_url, validation_file_name)
zip_ref = zipfile.ZipFile(validation_file_name, 'r')
zip_ref.extractall(validation_dir)
zip_ref.close()

HTTPError: ignored

In [ ]:
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(
validation_dir,
target_size=(150, 150),
batch_size = 100,
class_mode='binary'
)

Found 256 images belonging to 2 classes.


In [54]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if (logs.get('acc') is not None and logs.get('acc')>0.95 and logs.get('val_acc')>0.9):
      print("\nReached 95% accuracy so cancelling training!")
      self.model.stop_training = True

In [55]:
callbacks = myCallback()

In [56]:
history = model.fit(
train_generator,
epochs=15, callbacks = [callbacks]
)

Epoch 1/15
26/26 [==============================] - 105s 4s/step - loss: 2.5463e-06 - accuracy: 1.0000
Epoch 2/15
26/26 [==============================] - 95s 4s/step - loss: 1.4519e-06 - accuracy: 1.0000
Epoch 3/15
26/26 [==============================] - 93s 4s/step - loss: 1.0043e-06 - accuracy: 1.0000
Epoch 4/15
26/26 [==============================] - 95s 4s/step - loss: 9.9200e-07 - accuracy: 1.0000
Epoch 5/15
26/26 [==============================] - 101s 4s/step - loss: 7.2342e-07 - accuracy: 1.0000
Epoch 6/15
26/26 [==============================] - 99s 4s/step - loss: 5.7272e-07 - accuracy: 1.0000
Epoch 7/15
26/26 [==============================] - 92s 4s/step - loss: 5.2655e-07 - accuracy: 1.0000
Epoch 8/15
26/26 [==============================] - 93s 4s/step - loss: 4.4362e-07 - accuracy: 1.0000
Epoch 9/15
26/26 [==============================] - 94s 4s/step - loss: 4.5204e-07 - accuracy: 1.0000
Epoch 10/15
26/26 [==============================] - 92s 4s/step - loss: 5.3289e

In [ ]:
model.save('./Image_horsehuman.h5')

In [ ]:
model = tf.keras.models.load_model('./Image_horsehuman.h5')

In [ ]:
import os
print(os.stat('./Image_horsehuman.h5'))

os.stat_result(st_mode=33188, st_ino=4458029, st_dev=38, st_nlink=1, st_uid=0, st_gid=0, st_size=163033000, st_atime=1651765545, st_mtime=1651765545, st_ctime=1651765545)


In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np
path='./horse-or-human/training/humans/human01-11.png'
img = image.load_img(path, target_size=(150,150))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
image_tensor = np.vstack([x])
classes = model.predict(image_tensor)

1/1 [==============================] - 0s 18ms/step


In [ ]:
print(classes[0])

[1.]


Below is another model with transfer learning.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#drive.flush_and_unmount()

Mounted at /content/drive


In [ ]:
weights_file = "./drive/MyDrive/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.applications.inception_v3 import InceptionV3
pre_trained_model = InceptionV3(input_shape=(150, 150, 3),
include_top=False,
weights=None)
pre_trained_model.load_weights(weights_file)

In [ ]:
pre_trained_model.summary()

In [ ]:
for layer in pre_trained_model.layers:
  layer.trainable = False
last_layer = pre_trained_model.get_layer('mixed3')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

last layer output shape:  (None, 7, 7, 768)


In [ ]:
from keras.models import Model
from keras.layers import Flatten, Dense

In [ ]:
# Flatten the output layer to 1 dimension
x = Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = Dense(512, activation='relu')(x)
# Add a final sigmoid layer for classification
x = Dense(1, activation='sigmoid')(x)

In [ ]:
model = Model(pre_trained_model.input, x)
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-6), loss='binary_crossentropy',
metrics=['acc'])

In [49]:
history = model.fit(train_generator,
                    steps_per_epoch=11,
                    epochs=10,
                    verbose=2)

Epoch 1/10
11/11 - 51s - loss: 0.0812 - accuracy: 0.9673 - 51s/epoch - 5s/step
Epoch 2/10
11/11 - 40s - loss: 1.0317e-04 - accuracy: 1.0000 - 40s/epoch - 4s/step
Epoch 3/10
11/11 - 40s - loss: 6.8624e-05 - accuracy: 1.0000 - 40s/epoch - 4s/step
Epoch 4/10
11/11 - 40s - loss: 4.1287e-05 - accuracy: 1.0000 - 40s/epoch - 4s/step
Epoch 5/10
11/11 - 41s - loss: 1.5510e-05 - accuracy: 1.0000 - 41s/epoch - 4s/step
Epoch 6/10
11/11 - 41s - loss: 1.3383e-05 - accuracy: 1.0000 - 41s/epoch - 4s/step
Epoch 7/10
11/11 - 41s - loss: 8.4329e-06 - accuracy: 1.0000 - 41s/epoch - 4s/step
Epoch 8/10
11/11 - 41s - loss: 6.7434e-06 - accuracy: 1.0000 - 41s/epoch - 4s/step
Epoch 9/10
11/11 - 41s - loss: 3.7471e-06 - accuracy: 1.0000 - 41s/epoch - 4s/step
Epoch 10/10
11/11 - 40s - loss: 2.0543e-06 - accuracy: 1.0000 - 40s/epoch - 4s/step
